### Data Loading

In [22]:
import pandas as pd

df = pd.read_csv("salaries.csv")
df.sample(5)

,company,job,degree,salary_more_then_100k
12,facebook,business manager,bachelors,1
8,abc pharma,business manager,bachelors,0
0,google,sales executive,bachelors,0
3,google,business manager,masters,1
5,google,computer programmer,masters,1


In [23]:
df.shape

(16, 4)

### Feature Engineering: Label and One-Hot Encoding

In [24]:
df.degree.unique()

array(['bachelors', 'masters'], dtype=object)

In [25]:
df['degree_number'] = df['degree'].map({'bachelors': 1, 'masters':2})
df.head()

,company,job,degree,salary_more_then_100k,degree_number
0,google,sales executive,bachelors,0,1
1,google,sales executive,masters,0,2
2,google,business manager,bachelors,1,1
3,google,business manager,masters,1,2
4,google,computer programmer,bachelors,0,1


In [26]:
df.drop('degree', axis="columns", inplace=True)
df.head()

,company,job,salary_more_then_100k,degree_number
0,google,sales executive,0,1
1,google,sales executive,0,2
2,google,business manager,1,1
3,google,business manager,1,2
4,google,computer programmer,0,1


In [27]:
df_encoded = pd.get_dummies(df, columns=['company', 'job'], drop_first=True)
df_encoded.head()

,salary_more_then_100k,degree_number,company_facebook,company_google,job_computer programmer,job_sales executive
0,0,1,False,True,False,True
1,0,2,False,True,False,True
2,1,1,False,True,False,False
3,1,2,False,True,False,False
4,0,1,False,True,True,False


### Model Training: Using GINI Impurity

In [33]:
from sklearn.tree import DecisionTreeClassifier


X = df_encoded.drop('salary_more_then_100k', axis="columns")
y = df_encoded['salary_more_then_100k']

model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier()

In [34]:
from sklearn.metrics import classification_report

y_pred = model.predict(X)

report = classification_report(y, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [35]:
from sklearn.tree import export_text
print(export_text(model, feature_names=list(X.columns)))

|--- company_facebook <= 0.50
|   |--- job_sales executive <= 0.50
|   |   |--- degree_number <= 1.50
|   |   |   |--- company_google <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- company_google >  0.50
|   |   |   |   |--- job_computer programmer <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- job_computer programmer >  0.50
|   |   |   |   |   |--- class: 0
|   |   |--- degree_number >  1.50
|   |   |   |--- class: 1
|   |--- job_sales executive >  0.50
|   |   |--- class: 0
|--- company_facebook >  0.50
|   |--- class: 1



### Model Training: Using ENTROPY

In [36]:
from sklearn.tree import DecisionTreeClassifier


X = df_encoded.drop('salary_more_then_100k', axis="columns")
y = df_encoded['salary_more_then_100k']

model = DecisionTreeClassifier(criterion='entropy')
model.fit(X, y)

DecisionTreeClassifier(criterion='entropy')

In [37]:
from sklearn.metrics import classification_report

y_pred = model.predict(X)

report = classification_report(y, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [38]:
from sklearn.tree import export_text
print(export_text(model, feature_names=list(X.columns)))

|--- company_facebook <= 0.50
|   |--- job_sales executive <= 0.50
|   |   |--- degree_number <= 1.50
|   |   |   |--- job_computer programmer <= 0.50
|   |   |   |   |--- company_google <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- company_google >  0.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- job_computer programmer >  0.50
|   |   |   |   |--- class: 0
|   |   |--- degree_number >  1.50
|   |   |   |--- class: 1
|   |--- job_sales executive >  0.50
|   |   |--- class: 0
|--- company_facebook >  0.50
|   |--- class: 1



### Pedictions

In [41]:
X.head(2)

,degree_number,company_facebook,company_google,job_computer programmer,job_sales executive
0,1,False,True,False,True
1,2,False,True,False,True


In [42]:
model.predict([
    [1, 1, 0, 1, 0],
    [1, 0, 1, 0, 1]
])

C:\Users\alili\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1, 0])